# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [1]:
! pip install -U langchain

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/990.3 kB ? eta -:--:--
     ----------- -------------------------- 286.7/990.3 kB 5.9 MB/s eta 0:00:01
     --------------------------- ---------- 727.0/990.3 kB 7.6 MB/s eta 0:00:01
     -------------------------------------- 990.3/990.3 kB 7.8 MB/s eta 0:00:00
     ---------------------------------------- 0.0/374.2 kB ? eta -:--:--
     ------------------------------------- 374.2/374.2 kB 22.7 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.22
    Uninstalling langchain-core-0.2.22:
      Successfully uninstalled langchain-core-0.2.22
  Attempting uninstall: langchain
    Found existing installation: langchain 0.2.10
    Uninstalling langchain-0.2.10:
      Successfully uninstalled langchain-0.2.10


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.1.7 requires openai<2.0.0,>=1.24.0, but you have openai 1.14.2 which is incompatible.


## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [22]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [23]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""


biology_template = """你是一位经验丰富的生物学教师。
你对生物学的各个分支都有深入的了解，特别擅长用直观的方式解释复杂的生物学概念。
当遇到你不确定的问题时，你会谨慎地表示需要进一步查证。

这是一个问题：
{input}"""


computer_science_template = """你是一位资深的计算机科学专家。
你对编程、算法、数据结构和软件工程有深刻的理解。
你能够清晰地解释技术概念，并能提供详细的解决步骤和示例代码。
如果碰到超出你知识范围的问题，你会诚实地说明并建议可能的研究方向。

这是一个问题：
{input}"""


chinese_literature_template = """你是一位对汉语言文学充满热情的教授。
你对古典文学、现代文学和作文技巧有深入的研究。
你能够用生动的语言分析文学作品，并能引导学生发现语言的美。
在遇到需要更多考证的问题时，你会恭敬地表示这需要更多的文献支持。

这是一个问题：
{input}"""

In [24]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
    {
        "name": "生物",
        "description": "适用于回答生物学问题",
        "prompt_template": biology_template,
    },
    {
        "name": "计算机",
        "description": "适用于回答计算机科学问题",
        "prompt_template": computer_science_template,
    },
    {
        "name": "汉语言文学",
        "description": "适用于回答汉语言文学问题",
        "prompt_template": chinese_literature_template,
    },
]

In [25]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct",
             base_url='https://api.xiaoai.plus/v1')

In [26]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

In [27]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [28]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [29]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [30]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物: 适用于回答生物学问题', '计算机: 适用于回答计算机科学问题', '汉语言文学: 适用于回答汉语言文学问题']


In [31]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物学问题
计算机: 适用于回答计算机科学问题
汉语言文学: 适用于回答汉语言文学问题


In [32]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [33]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物学问题
计算机: 适用于回答计算机科学问题
汉语言文学: 适用于回答汉语言文学问题

<< INPUT >>
{input}

In [34]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [35]:
print(chain.invoke("堆栈是什么？?"))



> Entering new MultiPromptChain chain...
计算机: {'input': '什么是栈？'}
> Finished chain.
{'input': '什么是栈？', 'text': '\n\n栈是一种数据结构，它遵循"后进先出"(LIFO)的原则。这意味着最后进入栈的元素将第一个被移除。栈有两个基本操作：压入（push）和弹出（pop）。压入操作将一个新元素添加到栈的顶部，弹出操作将栈顶的元素移除。栈可以用来解决许多问题，比如括号匹配、函数调用和浏览器的历史记录。'}


In [36]:
print(
    chain.invoke(
        "文学是什么？"
    )
)



> Entering new MultiPromptChain chain...
汉语言文学: {'input': '文学是什么?'}
> Finished chain.
{'input': '文学是什么?', 'text': '\n\n文学是指用语言来表达思想、情感和想象力的艺术形式。它包括古典文学、现代文学、民间文学等不同类型，涵盖诗歌、散文、小说、戏剧等多种文学体裁。文学作品可以通过语言的艺术表现力来反映社会现实、探究人生道理、抒发情感等。它不仅是一种艺术形式，也是一种文化遗产，能够传承和影响人类的思想和价值观。'}


In [37]:
router_chain.verbose = True

In [38]:
print(chain.invoke("细胞体是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
生物: {'input': '细胞体是什么？'}
> Finished chain.
{'input': '细胞体是什么？', 'text': '\n\n细胞体是细胞内部包含细胞核、质壁、细胞质、细胞器等结构的总称，它是细胞生命活动的基本单位，也是生物体内所有生物活动的基础。细胞体由原生质和细胞壁组成，原生质是细胞内的液态物质，包含有大量的有机物和无机盐，细胞壁则是包裹在原生质表面的一层薄膜，起着保护和支持细胞的作用。细胞体的大小、形状和结构都因不同的生物种类和细胞类型而有所不同。'}


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains